In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
clientes = pd.read_csv('caso_estudo_clientes.csv', sep=';')
vendas = pd.read_csv('caso_estudo_vendas.csv', sep=';')
produtos = pd.read_csv('caso_estudo_produtos.csv', sep=';')
pagamentos = pd.read_csv('caso_estudo_pagamentos.csv', sep=';')
lojas = pd.read_csv('caso_estudo_lojas.csv', sep=';')

In [ ]:
# === LIMPEZA ===

#Ajustando dados nulos
clientes.loc[clientes.nome.isnull(),'nome'] = 'Sem Nome'
clientes.loc[clientes.sexo.isnull(),'sexo'] = 'o'
clientes.loc[clientes.dt_nasc.isnull(),'dt_nasc'] = '1/24/2023'
produtos.loc[9,'valor'] = 3500
produtos.loc[9,'produto'] = 'Monitor Gamer'

#Ajustando Datas
clientes.dt_nasc = pd.to_datetime(clientes.dt_nasc, format='%m/%d/%Y')
vendas.dt_venda = pd.to_datetime(vendas.dt_venda, format='%d/%m/%Y')
pagamentos.dt_pgto = pd.to_datetime(pagamentos.dt_pgto, format='%d/%m/%Y')

#Índices
clientes = clientes.set_index('id')
vendas = vendas.set_index('id')
produtos = produtos.set_index('id')
pagamentos = pagamentos.set_index('id')
lojas = lojas.set_index('id')


In [9]:
# === FUSION ===

lista_geral = vendas.join(clientes.add_prefix('cliente_'), on='id_cliente')
lista_geral = lista_geral.join(produtos.add_prefix('produto_'), on='id_produto')
lista_geral = lista_geral.join(lojas.add_prefix('loja_'), on='id_loja')
lista_geral = lista_geral.join(pagamentos.set_index('id_venda'))

NameError: name 'vendas' is not defined

In [7]:
# === FEATURE ENGINEERING ===

# Compras sem pagamento
lista_geral['pagamento'] = 1
lista_geral.loc[lista_geral.dt_pgto.isnull(), 'pagamento'] = 0

# Tempo entre compra e pagamento
lista_geral['tempo_pagamento'] = (lista_geral.dt_pgto - lista_geral.dt_venda).dt.days

# Idade dos Clientes
lista_geral['cliente_idade'] = np.floor((pd.to_datetime('today') - lista_geral.cliente_dt_nasc)/np.timedelta64(1, 'Y'))

NameError: name 'lista_geral' is not defined

# Análise - parte 01

Perguntas para análise:
- Quais lojas mais vendem?
- Quais produtos mais vendem?
- Quais lojas geram maior receita?
- Quais produtos geram maior receita?

## Lojas que mais vendem

In [6]:
lista_geral

NameError: name 'lista_geral' is not defined

In [ ]:
"""
Primeiro passo: Definir a coluna das lojas - loja_cidade
Segundo passo: Agrupar os dados nos valores desta coluna
obs: Não imprime os valores, pq retorna uma lista de dataframes (1 por cidade)
"""

lista_geral.groupby('loja_cidade')


In [5]:
"""
Quando quer retornar apenas a quantidade de vendas ("que mais vendem")
se usa a função count()
"""

lista_geral.groupby('loja_cidade').count()

NameError: name 'lista_geral' is not defined

In [ ]:
lista_geral.groupby('loja_cidade').count().produto_produto

In [ ]:
"""
Para ordenar a contagem do dataframe, se usa a função sort_values()
"""
lista_geral.groupby('loja_cidade').count().produto_produto.sort_values()

In [19]:
"""
Para ordenar de forma inversa (do maior para menor), basta mudar para
false a propriedade ascending
"""
lista_geral.groupby('loja_cidade').count().produto_produto.sort_values(ascending=False)

loja_cidade
Belo Horizonte    118
São Paulo         114
Recife            110
Palmas            107
Curitiba           99
Salvador           99
Vitória            95
Santa Catarina     92
Brasília           83
Rio de Janeiro     83
Name: produto_produto, dtype: int64

In [2]:
"""
CURIOSIDADE: se desejar ver as 3 cidades que mais venderam (top 3)
 se usa a função head()
"""

lista_geral.groupby('loja_cidade').count().produto_produto.sort_values(ascending=False).head(3)

NameError: name 'lista_geral' is not defined

In [4]:
"""
Gerando gráfico de barras, primeiro passa o eixo x (lojas-index)
e depois o eixo y (contagem-values)
"""

grafico_lojas = lista_geral.groupby('loja_cidade').count().produto_produto.sort_values(ascending=False)

plt.bar(grafico_lojas.index, grafico_lojas.values)
plt.title('Quantidade de Vendas por Loja')
plt.show()

NameError: name 'lista_geral' is not defined

In [3]:
"""
A função xticks rotaciona as legendas do eixo x
"""

grafico_lojas = lista_geral.groupby('loja_cidade').count().produto_produto.sort_values(ascending=False)

plt.bar(grafico_lojas.index, grafico_lojas.values)
plt.title('Quantidade de Vendas por Loja')
plt.xticks(rotation=45)
plt.show()

NameError: name 'lista_geral' is not defined

In [ ]:
"""
Para aumentar proporcionalmente (largura x altura)
o tamanho do gráfico, se usa a função figure() junto com
a propriedade figsize

"""

grafico_lojas = lista_geral.groupby('loja_cidade').count().produto_produto.sort_values(ascending=False)

plt.figure(figsize=(13,5))
plt.bar(grafico_lojas.index, grafico_lojas.values)
plt.title('Quantidade de Vendas por Loja')
plt.xticks(rotation=45)
plt.show()

## Produtos que mais vendem

In [ ]:
grafico_lojas = lista_geral.groupby('loja_cidade').count().produto_produto.sort_values(ascending=False)

plt.figure(figsize=(13,5))
plt.bar(grafico_lojas.index, grafico_lojas.values)
plt.title('Produtos mais vendidos')
plt.xticks(rotation=45)
plt.show()

## Quais lojas geram maior receita?

## Quais produtos geram maior receita?